In [ ]:
import cv2
import random
import numpy as np

In [ ]:
import zipfile
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/

Mounted at /content/drive
/content/drive


In [ ]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/driving_dataset.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [ ]:
xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0
test_batch_pointer = 0

#read data.txt
with open("/tmp/driving_dataset/data.txt") as f:
    for line in f:
        xs.append("/tmp/driving_dataset/" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * 3.14159265 / 180)

#get number of images
num_images = len(xs)

#shuffle list of images
c = list(zip(xs, ys))
random.shuffle(c)
xs, ys = zip(*c)
# print(xs[0])

train_xs = xs[:int(len(xs) * 0.8)]
# print(train_xs[0])
train_ys = ys[:int(len(xs) * 0.8)]

val_xs_2 = xs[-int(len(xs) * 0.2):]
# print(val_xs[0])
val_ys_2 = ys[-int(len(xs) * 0.2):]


# num_val_images = len(val_xs_2)
# print(num_val_images)

val_xs = val_xs_2[:int(len(val_xs_2) * 0.5)]
val_ys = val_ys_2[:int(len(val_ys_2) * 0.5)]

test_xs = val_xs_2[-int(len(val_xs_2) * 0.5):]
test_ys = val_ys_2[-int(len(val_ys_2) * 0.5):]

num_train_images = len(train_xs)
print(num_train_images)

num_val_images = len(val_xs)
print(num_val_images)

num_test_images = len(test_xs)
print(num_test_images)

print(train_xs[0])
print(val_xs[0])
print(test_xs[0])

print(train_ys[0])
print(val_ys[0])
print(test_ys[0])
# l1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
# print(l1)
# train_dt = l1[:int(len(l1) * 0.8)]
# val_dt = l1[-int(len(l1) * 0.2):]
# print(train_dt)
# print(val_dt)
# new_val_dt = val_dt[:int(len(val_dt) * 0.5)]
# test_dt = val_dt[-int(len(val_dt) * 0.5):]

# print('testing the division:')
# print(new_val_dt)
# print(test_dt)

36324
4540
4540
/tmp/driving_dataset/34789.jpg
/tmp/driving_dataset/26985.jpg
/tmp/driving_dataset/18081.jpg
0.017627825425000004
0.022863813175
0.13020156205


In [ ]:
num_train_images

36324

In [ ]:
num_val_images

4540

In [ ]:
num_test_images

4540

In [ ]:
def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
      x_out.append(cv2.resize(cv2.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], (200, 66)) / 255.0)
      y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
      x_out.append(cv2.resize(cv2.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], (200, 66)) / 255.0)
      y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    # print(x_out)
    # print(y_out)
    return x_out, y_out

def LoadTestBatch(batch_size):
    global test_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
      x_out.append(cv2.resize(cv2.imread(test_xs[(test_batch_pointer + i) % num_test_images])[-150:], (200, 66)) / 255.0)
      y_out.append([test_ys[(test_batch_pointer + i) % num_test_images]])
    test_batch_pointer += batch_size
    # print(x_out)
    # print(y_out)
    return x_out, y_out

In [ ]:
LoadTestBatch(3)

([array([[[0.18431373, 0.26666667, 0.24313725],
          [0.16078431, 0.27058824, 0.22352941],
          [0.17254902, 0.24705882, 0.23137255],
          ...,
          [0.09411765, 0.09803922, 0.08235294],
          [0.08235294, 0.08627451, 0.07058824],
          [0.07843137, 0.07843137, 0.0627451 ]],
  
         [[0.27843137, 0.30196078, 0.3254902 ],
          [0.23529412, 0.28627451, 0.25490196],
          [0.26666667, 0.31372549, 0.28627451],
          ...,
          [0.09411765, 0.10980392, 0.08627451],
          [0.09019608, 0.09803922, 0.0745098 ],
          [0.07058824, 0.0745098 , 0.05098039]],
  
         [[0.28627451, 0.31372549, 0.3254902 ],
          [0.3254902 , 0.34117647, 0.36078431],
          [0.31764706, 0.35294118, 0.36470588],
          ...,
          [0.09411765, 0.10196078, 0.07843137],
          [0.09411765, 0.10196078, 0.07843137],
          [0.08627451, 0.08627451, 0.0627451 ]],
  
         ...,
  
         [[0.30196078, 0.24313725, 0.24705882],
          [0.3